<a href="https://colab.research.google.com/github/dani-lbnl/mudit/blob/main/Topic_Modeling_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install packages

In [2]:
!pip install -U kaleido
!pip install bertopic
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 79.9 MB 98 kB/s 
     |████████████████████████████████| 57 kB 4.6 MB/s 
     |████████████████████████████████| 88 kB 6.5 MB/s 
     |████████████████████████████████| 79 kB 10.8 MB/s 
     |████████████████████████████████| 5.2 MB 66.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.0 MB 53.6 MB/s 
     |████████████████████████████████| 1.2 MB 50.0 MB/s 
     |████████████████████████████████| 77 kB 8.5 MB/s 
     |████████████████████████████████| 895 kB 54.2 MB/s 
     |████████████████████████████████| 636 kB 41.4 MB/s 
     |████████████████████████████████| 6.6 MB 44.1 MB/s 
     |████████████████████████████████| 1.1 MB 49.9 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2330765 sha256=f31f8b7b839a35662af72ce72fca2f58560ed1031c1053619a93d5acb51479f4
  Stored in director

In [3]:
# from top2vec import Top2Vec
import pandas as pd
import numpy as np
import multiprocessing
import time
from google.colab import drive
import os
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

import re
import string

import nltk
nltk.download('stopwords')
import spacy

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import kaleido

multiprocessing.cpu_count()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('wordnet')
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer:
  def __init__(self):
    self.wnl = WordNetLemmatizer()
  def __call__(self, doc):
    return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
# # using spacy

# nlp = spacy.load('en_core_web_lg')

# class LemmaTokenizer:
#   def __init__(self):
#     self.wnl = WordNetLemmatizer()
#   def __call__(self, doc):
#     # return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

#     return [self.wnl.t.lemma_ for t in nlp(doc)]

OSError: ignored

## Define path of files

In [10]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# read all files
path = '/content/drive/MyDrive/NLP/ALS Spreadsheets/'
files = os.listdir(path)
files.sort()
files
# beamline 2.4 bad ..removed; also one more check which one
# 3.2.1 bad

['Beamline_1.4.xls',
 'Beamline_10.0.1.xls',
 'Beamline_10.3.1.xls',
 'Beamline_10.3.2.xls',
 'Beamline_11.0.1.xls',
 'Beamline_11.0.2.xls',
 'Beamline_11.3.1.xls',
 'Beamline_11.3.2.xls',
 'Beamline_12.0.1.1.xls',
 'Beamline_12.0.1.2.xls',
 'Beamline_12.0.1.4.xls',
 'Beamline_12.0.2.xls',
 'Beamline_12.2.1.xls',
 'Beamline_12.2.2.xls',
 'Beamline_12.3.1.xls',
 'Beamline_12.3.2.xls',
 'Beamline_2.1.xls',
 'Beamline_3.1.1.xls',
 'Beamline_3.3.2.xls',
 'Beamline_4.0.2.xls',
 'Beamline_4.0.3.1.xls',
 'Beamline_4.0.3.2.xls',
 'Beamline_4.2.2.xls',
 'Beamline_5.0.1.xls',
 'Beamline_5.0.2.xls',
 'Beamline_5.0.3.xls',
 'Beamline_5.3.1.xls',
 'Beamline_5.3.2.1.xls',
 'Beamline_5.3.2.2.xls',
 'Beamline_5.4.xls',
 'Beamline_6.0.1.xls',
 'Beamline_6.0.2.xls',
 'Beamline_6.1.2.xls',
 'Beamline_6.3.1.xls',
 'Beamline_6.3.2.xls',
 'Beamline_7.0.1.xls',
 'Beamline_7.0.2.xls',
 'Beamline_7.3.1.xls',
 'Beamline_7.3.3.xls',
 'Beamline_8.0.1.xls',
 'Beamline_8.2.1.xls',
 'Beamline_8.2.2.xls',
 'Beamline_

In [17]:
files[-5]

'Beamline_8.3.1.xls'

## Helper Functions

In [23]:
def create_input(df):

  # for title
  indices = df['Title'].isna()
  df.loc[indices,'Title'] = ""

  # for abstract
  indices = df['Abstract'].isna()
  df.loc[indices,'Abstract'] = ""

  # combined - title + abstract
  df['Combined'] = df['Title'] + " " + df['Abstract']
  combined = list(df['Combined'])

  # remove elements with blank
  while " " in combined:
    combined.remove(" ")

  # remove patterns
  pattern = r'<inf>|</inf>|<sup>|</sup>|inf|/inf'
  comb_clean = []
  for l in combined:
    mod_string = re.sub(pattern, ' ', l )
    comb_clean.append(mod_string)


  return comb_clean

In [ ]:
# def lemma_spacy(df_combined):

#   new_df= []
#   for doc in df_combined:
#     filt_combined = []
#     for word in nlp(doc):
#       filt_combined.append(word.lemma_)

#     new_df.append(" ".join(filt_combined))

#   return new_df

  

In [25]:
def remove_stop(df_combined):

  # remove stopwords

  new_df= []
  for doc in df_combined:
    filt_combined = []
    for word in word_tokenize(doc):

      if word.lower() not in stopwords.words('english'):
        # print(word)
        if word.lower() == "perovskites":
          filt_combined.append("perovskite")
        else:
          filt_combined.append(word)
    new_df.append(" ".join(filt_combined))

  return new_df

In [26]:
def berttopic_basic(df):

  topic_model = BERTopic()
  topics, probs = topic_model.fit_transform(df)

  # added lemmatization
  # update using lemmatization

  tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
  # vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
  topic_model.update_topics(df, topics, vectorizer_model=tf_vectorizer)

  # topic_info_df = pd.DataFrame(topic_model.get_topic_info())

  return topic_model

In [27]:
import joblib
import os 

def create_path_if_not_exists(datapath):
    '''Create the new file if not exists and save the data'''

    if not os.path.exists(datapath):
        os.makedirs(datapath) 

## example

In [6]:
df = pd.read_table(path + "Beamline_7.3.3.xls", on_bad_lines='skip')

# combine abstract and title
input_data = create_input(df)

# remove stopwords
filtered_input = remove_stop(input_data)

NameError: ignored

## Using BERTopic

In [ ]:
start_time = time.time()
iter_version = "BERTopic_v2/"


for i in range(len(files)):
  print("File:",str(i+1) + ' ' + files[i])
  beam_name = files[i].split('xls')[0][:-1]

  df = pd.read_table(path + files[i], on_bad_lines='skip')


  # combine abstract and title
  input_data = create_input(df)

  # remove stopwords
  filtered_input = remove_stop(input_data)

  # run topic modeling algorithm
  model = berttopic_basic(filtered_input)

  base_path = '/content/drive/MyDrive/NLP/'
  model_path = base_path + iter_version + beam_name + "/"
  create_path_if_not_exists(model_path)

  # save model as pickle file
  file_path = model_path + "model" + beam_name + ".pkl"
  joblib.dump(model, file_path) 

  # visualize as html
  fig = model.visualize_barchart(top_n_topics = len(model.topics))
  fig_name = model_path + "bar_chart" + beam_name  +".html"
  fig_name_png = model_path + "bar_chart" + beam_name  +".png"

  fig.write_html(fig_name)
  fig.write_image(fig_name_png)


  # save as excel
  # df_info = pd.DataFrame(model.get_topic_info())
  excel_name = model_path + "Topic_Results.xlsx"
  # df_info.to_excel(excel_name,sheet_name="topic_info")

  df_topics = pd.DataFrame(model.topics)
  df_topics.to_excel(excel_name,sheet_name="topic_words")

  print("File: {} done ".format(i+1) )


print('Total time taken (mins): ', int((time.time()-start_time)/60))


File: 1 Beamline_1.4.xls


/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


File: 1 done 
File: 2 Beamline_10.0.1.xls
File: 2 done 
File: 3 Beamline_10.3.1.xls
File: 3 done 
File: 4 Beamline_10.3.2.xls
File: 4 done 
File: 5 Beamline_11.0.1.xls
File: 5 done 
File: 6 Beamline_11.0.2.xls
File: 6 done 
File: 7 Beamline_11.3.1.xls
File: 7 done 
File: 8 Beamline_11.3.2.xls
File: 8 done 
File: 9 Beamline_12.0.1.1.xls
File: 9 done 
File: 10 Beamline_12.0.1.2.xls
File: 10 done 
File: 11 Beamline_12.0.1.4.xls
File: 11 done 
File: 12 Beamline_12.0.2.xls
File: 12 done 
File: 13 Beamline_12.2.1.xls
File: 13 done 
File: 14 Beamline_12.2.2.xls
File: 14 done 
File: 15 Beamline_12.3.1.xls
File: 15 done 
File: 16 Beamline_12.3.2.xls
File: 16 done 
File: 17 Beamline_2.1.xls
File: 17 done 
File: 18 Beamline_3.1.1.xls
File: 18 done 
File: 19 Beamline_3.3.2.xls
File: 19 done 
File: 20 Beamline_4.0.2.xls
File: 20 done 
File: 21 Beamline_4.0.3.1.xls
File: 21 done 
File: 22 Beamline_4.0.3.2.xls
File: 22 done 
File: 23 Beamline_4.2.2.xls
File: 23 done 
File: 24 Beamline_5.0.1.xls
File:

## Using SPECTER

In [20]:
def remove_punct(df):

  # remove punctuation
  text_col = [''.join(letter for letter in word if letter not in string.punctuation) for word in df]

  return text_col


In [51]:
def berttopic_specter(df):

  sentence_model = SentenceTransformer('allenai-specter')
  topic_model_specter = BERTopic(embedding_model=sentence_model)

  topics, probs = topic_model_specter.fit_transform(df)

  # added lemmatization
  # update using lemmatization


  tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
  # vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
  topic_model_specter.update_topics(df, topics, vectorizer_model=tf_vectorizer)

  if len(topic_model_specter.topics)==1:
    if -1 in list(topic_model_specter.topics.keys()):
      # retrain model 
      topic_model_specter = BERTopic(embedding_model=sentence_model,min_topic_size = 5)
      topics, probs = topic_model_specter.fit_transform(filtered_input_2)

      tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
      # vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
      topic_model_specter.update_topics(df, topics, vectorizer_model=tf_vectorizer)

  # topic_info_df = pd.DataFrame(topic_model.get_topic_info())

  return topic_model_specter

In [49]:
# df = pd.read_table(path + "Beamline_4.0.3.1.xls", on_bad_lines='skip')
# # df = pd.read_table(path + "Beamline_11.0.1.xls", on_bad_lines='skip')


# # # combine abstract and title
# # input_data = create_input(df)

# # # remove stopwords
# # filtered_input = remove_stop(input_data)

# # filtered_input_2 = remove_punct(filtered_input)

# sentence_model = SentenceTransformer('allenai-specter')
# topic_model_specter = BERTopic(embedding_model=sentence_model)

# topics, probs = topic_model_specter.fit_transform(filtered_input_2)

# print(topic_model_specter.topics)



# if len(topic_model_specter.topics)==1:
#   if -1 in list(topic_model_specter.topics.keys()):
#     # retrain model 
#     topic_model_specter = BERTopic(embedding_model=sentence_model,min_topic_size = 5)
#     topics, probs = topic_model_specter.fit_transform(filtered_input_2)

#     tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
#     # vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
#     topic_model_specter.update_topics(df, topics, vectorizer_model=tf_vectorizer)



# # run topic modeling algorithm
# # model = berttopic_specter(filtered_input_2)



{-1: [('ultrafast', 1.6801020957902655), ('small', 0.9526677587279724), ('repetition', 0.2620700961367291), ('khz', 0.2620700961367291), ('computers', 0.2620700961367291), ('mhz', 0.2501466367149557), ('efficient', 0.2370786223680821), ('energy', 0.23352710838437443), ('center', 0.22976590893320176), ('faster', 0.22523404980752915)], 0: [('solar', 0.03933516389276976), ('cells', 0.0379834677124056), ('polymer', 0.035142449804982635), ('organic', 0.028611481450864132), ('morphology', 0.026961603414980453), ('performance', 0.025014747584466004), ('efficiency', 0.02358458779618354), ('high', 0.022362935801789857), ('acceptor', 0.02228680688855267), ('molecular', 0.021089679284766703)], 1: [('magnetic', 0.10446360928999067), ('spin', 0.052660301488942074), ('artificial', 0.03177173293315308), ('state', 0.03109486516617621), ('temperature', 0.029942083407576685), ('domain', 0.029206302336515878), ('antiferromagnetic', 0.027065696542852014), ('fe', 0.02561574851377066), ('thin', 0.0247265035

In [53]:
start_time = time.time()
iter_version = "BERTopic_SPECTER_v1/"


for i in range(len(files)):
  print("File:",str(i+1) + ' ' + files[i])
  beam_name = files[i].split('xls')[0][:-1]
  

  df = pd.read_table(path + files[i], on_bad_lines='skip')


  # combine abstract and title
  input_data = create_input(df)

  # remove stopwords
  filtered_input = remove_stop(input_data)

  # remove punctutation
  filtered_input_2 = remove_punct(filtered_input)

  # run topic modeling algorithm
  model = berttopic_specter(filtered_input_2)

  base_path = '/content/drive/MyDrive/NLP/'
  model_path = base_path + iter_version + beam_name + "/"
  create_path_if_not_exists(model_path)

  # save model as pickle file
  file_path = model_path + "model" + beam_name + ".pkl"
  joblib.dump(model, file_path) 

  # visualize as html
  fig = model.visualize_barchart(top_n_topics = len(model.topics))
  fig_name = model_path + "bar_chart" + beam_name  +".html"
  fig_name_png = model_path + "bar_chart" + beam_name  +".png"

  fig.write_html(fig_name)
  fig.write_image(fig_name_png)


  # save as excel
  # df_info = pd.DataFrame(model.get_topic_info())
  excel_name = model_path + "Topic_Results.xlsx"
  # df_info.to_excel(excel_name,sheet_name="topic_info")

  df_topics = pd.DataFrame(model.topics)
  df_topics.to_excel(excel_name,sheet_name="topic_words")

  print("File: {} done ".format(i+1) )


print('Total time taken (mins): ', int((time.time()-start_time)/60))


File: 1 Beamline_1.4.xls
File: 1 done 
File: 2 Beamline_10.0.1.xls
File: 2 done 
File: 3 Beamline_10.3.1.xls
File: 3 done 
File: 4 Beamline_10.3.2.xls
File: 4 done 
File: 5 Beamline_11.0.1.xls
File: 5 done 
File: 6 Beamline_11.0.2.xls
File: 6 done 
File: 7 Beamline_11.3.1.xls
File: 7 done 
File: 8 Beamline_11.3.2.xls
File: 8 done 
File: 9 Beamline_12.0.1.1.xls
File: 9 done 
File: 10 Beamline_12.0.1.2.xls
File: 10 done 
File: 11 Beamline_12.0.1.4.xls
File: 11 done 
File: 12 Beamline_12.0.2.xls
File: 12 done 
File: 13 Beamline_12.2.1.xls
File: 13 done 
File: 14 Beamline_12.2.2.xls
File: 14 done 
File: 15 Beamline_12.3.1.xls
File: 15 done 
File: 16 Beamline_12.3.2.xls
File: 16 done 
File: 17 Beamline_2.1.xls
File: 17 done 
File: 18 Beamline_3.1.1.xls
File: 18 done 
File: 19 Beamline_3.3.2.xls
File: 19 done 
File: 20 Beamline_4.0.2.xls
File: 20 done 
File: 21 Beamline_4.0.3.1.xls
File: 21 done 
File: 22 Beamline_4.0.3.2.xls
File: 22 done 
File: 23 Beamline_4.2.2.xls
File: 23 done 
File: 24

## SCI BERT

In [ ]:
def berttopic_scibert(df):

  scibert_emb = TransformerDocumentEmbeddings('allenai/scibert_scivocab_uncased')
  topic_model_specter = BERTopic(embedding_model=sentence_model)

  topics, probs = topic_model_specter.fit_transform(df)

  # added lemmatization
  # update using lemmatization


  tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
  # vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
  topic_model_specter.update_topics(df, topics, vectorizer_model=tf_vectorizer)

  if len(topic_model_specter.topics)==1:
    if -1 in list(topic_model_specter.topics.keys()):
      # retrain model 
      topic_model_specter = BERTopic(embedding_model=sentence_model,min_topic_size = 5)
      topics, probs = topic_model_specter.fit_transform(filtered_input_2)

      tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
      # vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
      topic_model_specter.update_topics(df, topics, vectorizer_model=tf_vectorizer)

  # topic_info_df = pd.DataFrame(topic_model.get_topic_info())

  return topic_model_specter

In [ ]:
scibert_emb = TransformerDocumentEmbeddings('allenai/scibert_scivocab_uncased')


## getting most representative docs

In [66]:
!pip install flair!pip install flair

     |████████████████████████████████| 401 kB 7.4 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 1.6 MB 44.6 MB/s 
     |████████████████████████████████| 1.2 MB 47.3 MB/s 
     |████████████████████████████████| 19.7 MB 1.2 MB/s 
     |████████████████████████████████| 788 kB 51.9 MB/s 
     |████████████████████████████████| 46 kB 4.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 981 kB 39.0 MB/s 
     |████████████████████████████████| 48 kB 4.7 MB/s 
     |████████████████████████████████| 199 kB 45.9 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9692 sha256=5840b98f9c4f8e325e2c1d83f5087d4a12eb14c7c2ba5364837035bb89f52a7d
  Stored in directory: /root/.cache/pip/wheels/ba/e0/7e/726e872a53f7358b4b96a9975b04e98113b005cd8

In [67]:
# df = pd.read_table(path + "Beamline_4.0.3.1.xls", on_bad_lines='skip')
df = pd.read_table(path + "Beamline_8.3.2.xls", on_bad_lines='skip')


# combine abstract and title
input_data = create_input(df)

# remove stopwords
filtered_input = remove_stop(input_data)

filtered_input_2 = remove_punct(filtered_input)

# sentence_model = SentenceTransformer('allenai-specter')
from flair.embeddings import TransformerDocumentEmbeddings

scibert_emb = TransformerDocumentEmbeddings('allenai/scibert_scivocab_uncased')

topic_model_specter = BERTopic(embedding_model=scibert_emb)

topics, probs = topic_model_specter.fit_transform(filtered_input_2)

tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
# vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
topic_model_specter.update_topics(filtered_input_2, topics, vectorizer_model=tf_vectorizer)

# run topic modeling algorithm
# model = berttopic_specter(filtered_input_2)



Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

In [68]:
topic_model_specter.topics

{0: [('bone', 0.03063984255987109),
  ('xray', 0.02902255489675332),
  ('material', 0.023195228508882466),
  ('using', 0.02068183015038232),
  ('property', 0.020598374963194867),
  ('fracture', 0.01935784534218754),
  ('data', 0.01919034824212232),
  ('structure', 0.019063411338098232),
  ('study', 0.017373779554494498),
  ('result', 0.017121597856141854)],
 1: [('xray', 0.06049471826180843),
  ('xf', 0.05621979181854755),
  ('tomography', 0.05387446917707999),
  ('using', 0.051301877161793595),
  ('xylem', 0.047012763544214756),
  ('synchrotron', 0.04215640563399655),
  ('imaging', 0.03486103310194392),
  ('hydraulic', 0.03381740462441962),
  ('pd', 0.030342935629902444),
  ('study', 0.02975326080112758)],
 2: [('xylem', 0.05980960312393679),
  ('specie', 0.05211146222066246),
  ('vessel', 0.05111054803813634),
  ('water', 0.04829032719957237),
  ('leaf', 0.04725566760958499),
  ('embolism', 0.04530852489860163),
  ('plant', 0.0447798821123612),
  ('root', 0.04336304721214872),
  ('dr

In [61]:
topic_model_specter.topics

{-1: [('quenching', 0.039007124279581085),
  ('pumice', 0.03515689749860541),
  ('explosive', 0.03417091355633744),
  ('steel', 0.031205699423664867),
  ('using', 0.028379499355876),
  ('sample', 0.028353552749561115),
  ('quenched', 0.028154570846637698),
  ('concrete', 0.02693668663538385),
  ('corrosion', 0.02659793233068977),
  ('oil', 0.026284340352653462)],
 0: [('xray', 0.033514742601623945),
  ('material', 0.031681427645189435),
  ('data', 0.02517028298342875),
  ('composite', 0.02438876514374097),
  ('fiber', 0.023581111542990463),
  ('scaffold', 0.022191385136204535),
  ('tomography', 0.021792925439949908),
  ('mechanical', 0.02149342718186911),
  ('image', 0.020919124428688617),
  ('structure', 0.02070558903754865)],
 1: [('permeability', 0.028096895379658256),
  ('eruption', 0.027737694353649117),
  ('magma', 0.0259007981363251),
  ('pore', 0.023993954571015327),
  ('experiment', 0.023292833344230535),
  ('area', 0.02277748202080898),
  ('2', 0.022162644580630664),
  ('surf

In [62]:
topic_model_specter.get_representative_docs(2)

['Investigating Oxygen Content Porous Transport Layers PEM Electrolyzers Operando XRay Computed Tomography Machine Learning Hydrogen gas good candidate replace traditional  carbonbased energy carriers  easily transported  many vehicles already utilize hydrogenbased fuel cells power locomotion  produce hydrogen gas  water electrolysis goto electrochemical process  Although water electrolysis wellunderstood process  morphological transport processes inherent proton exchange membrane  PEM  electrolyzer affect efficiency water electrolysis PEM electrolyzer still relatively unexplored  order probe processes insitu  xray computed tomography radiography performed catalyst coated membranes  CCM  model electrolyzer different flowrates current densities determine different operating conditions alter oxygen production  slug bubble formation  dwell time porous transport layer  PTL   Xray computed tomography utilized create 3D images  radiography utilized visualize transient oxygen production dwell

In [63]:
df

,DOI,Title,Journal,Authors,Pub Year,Beamlines,Funding Source,Research Area,Pub TYpe,Citation Count,Abstract,Unnamed: 11,Combined
0,10.1016/j.compositesa.2017.12.024,Insights from in-situ x-ray computed tomograph...,Composites Part A: Applied Science and Manufac...,"Larson, Natalie M.,Zok, Frank ,",2018.0,8.3.2,"DOE, Basic Energy Sciences NSF Other U.S. Gove...",Materials - Condensed matter,Refereed Journal Article,27.0,,NaN,Insights from in-situ x-ray computed tomograph...
1,10.1016/j.bone.2004.07.016,Effect of aging on the toughness of human cort...,Bone,"Nalla, Ravi K.,Kruzic, J.J. ,Kinney, John H.,R...",2004.0,8.3.2,"DOE, Basic Energy Sciences U.S. National Insti...",None,Refereed Journal Article,193.0,Age-related deterioration of the fracture prop...,NaN,Effect of aging on the toughness of human cort...
2,10.1016/j.biomaterials.2004.09.004,Age related transparent Root Dentin: Mineral C...,Biomaterials,"Kinney, John H.,Nalla, Ravi K.,Pople, J.A. ,Br...",2005.0,8.3.2,"DOE, Basic Energy Sciences U.S. National Insti...",None,Refereed Journal Article,173.0,Many fractures occur in teeth that have been a...,NaN,Age related transparent Root Dentin: Mineral C...
3,10.1073/pnas.0507417102,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,
4,NaN,TGF-beta regulates the mechanical properties a...,Proceedings of the National Academy of Science...,"Balooch, Guive ,Balooch, Mehdi ,Nalla, Ravi K....",2005.0,8.3.2,"DOE, Basic Energy Sciences",Bioscience,Refereed Journal Article,169.0,,NaN,TGF-beta regulates the mechanical properties a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,10.1107/S160057752101287X,A wide-field micro-computed tomography detecto...,Journal of Synchrotron Radiation,"Yakovlev, M.A. ,Vanselow, D.J. ,Ngu, M.S. ,Zai...",2022.0,8.3.2,"DOE, Basic Energy Sciences NIH Other (Pennsylv...",Applied Science - techniques,Refereed Journal Article,0.0,Ideal three-dimensional imaging of complex sam...,NaN,A wide-field micro-computed tomography detecto...
507,NaN,Examining the effect of fire retardant on the ...,NaN,"Boigne, E. ,Bennett, N. ,Wang, A. ,Ihme, M. ,",2021.0,8.3.2,NSF,Energy Science,Non-refereed Publication,0.0,Fire retardants are used to slow down the spre...,NaN,Examining the effect of fire retardant on the ...
508,10.1038/s41598-021-96867-7,Quantitative X-ray tomographic analysis reveal...,Scientific Reports,"Li, Y. ,Parkinson, D.Y. ,Feng, J. ,Xia, C.-h. ...",2021.0,8.3.2,"DOE, Basic Energy Sciences NIH Foreign (Tsingh...",Bioscience,Refereed Journal Article,1.0,"Cataracts, named for pathological light scatte...",NaN,Quantitative X-ray tomographic analysis reveal...
509,NaN,Mechanisms of Pierce's Disease Resistance in G...,NaN,"Fanton Borges, A.C. ,",2021.0,8.3.2,Other (Yale Institute for Biospheric Studies),Bioscience,Thesis,0.0,Xylella fastidiosa (Xf) is a xylem-dwelling ba...,NaN,Mechanisms of Pierce's Disease Resistance in G...
